In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

import statsmodels.api as sm

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
N = 10000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 1.03212278, -2.01320535,  0.96700712])

In [5]:
model.parameter_covariance_

array([[  7.00765966e-04,  -6.04389637e-04,  -6.04824497e-04],
       [ -6.04389637e-04,   1.19436585e-03,   2.58409084e-05],
       [ -6.04824497e-04,   2.58409084e-05,   1.18456813e-03]])

In [6]:
np.sqrt(model.parameter_covariance_)

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


array([[ 0.02647198,         nan,         nan],
       [        nan,  0.0345596 ,  0.0050834 ],
       [        nan,  0.0050834 ,  0.03441756]])

In [7]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     2128.
Date:                Sat, 26 Aug 2017   Prob (F-statistic):               0.00
Time:                        20:20:12   Log-Likelihood:                -14147.
No. Observations:               10000   AIC:                         2.830e+04
Df Residuals:                    9997   BIC:                         2.832e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0321      0.026     38.989      0.0

## Linear Model with Sample Weights

In [8]:
sample_weights = np.random.uniform(0, 2, size=N)

In [9]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [10]:
model.coef_

array([ 1.04952587, -2.01690863,  0.95049089])

## Logistic Model

In [32]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [33]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [34]:
model.coef_

array([ 0.87369776, -2.30124442,  1.79603592])

In [35]:
model.dispersion_

array(1.0)

In [36]:
model.parameter_covariance_

array([[  1.87181978e-02,  -2.80152340e-02,   8.07656574e-04],
       [ -2.80152340e-02,   1.11936850e+00,  -1.19006455e+00],
       [  8.07656574e-04,  -1.19006455e+00,   1.32653709e+00]])

In [37]:
np.sqrt(np.diag(model.parameter_covariance_) / model.dispersion_)

array([ 0.13681446,  1.05800212,  1.15175392])

In [38]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.655149
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Sat, 26 Aug 2017   Pseudo R-squ.:                0.008927
Time:                        20:21:46   Log-Likelihood:                -655.15
converged:                       True   LL-Null:                       -661.05
                                        LLR p-value:                  0.002736
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8737      0.137      6.386      0.000       0.606       1.142
x1            -2.3012      1.

In [39]:
coefs = np.empty((1000, 3))
for i in range(1000):
    y_logistic = np.random.binomial(1, p=p, size=N)
    model = GLM(family=Bernoulli())
    model.fit(X, y_logistic)
    coefs[i, :] = model.coef_
    
print(coefs.std(axis=0))

[ 0.14375849  1.0940713   1.18051492]


## Poission Model

In [21]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [22]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [23]:
model.coef_

array([ 0.95991397, -1.99216788,  1.04996243])

## Poisson with Exposures

In [24]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [25]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [26]:
model.coef_

array([ 1.00413368, -1.98361939,  0.97751309])

## Linear Model with Correlated Predictors

In [27]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.9*X[:, 1] + np.random.uniform(-0.1, 0.1, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [28]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [29]:
model.coef_

array([ 1.03681471, -1.8754879 ,  0.83916954])

In [30]:
model.parameter_covariance_

array([[ 0.0042682 , -0.00575067, -0.00061137],
       [-0.00575067,  0.2667787 , -0.2843184 ],
       [-0.00061137, -0.2843184 ,  0.31763773]])